In [1]:
# import the libraries you'll need to work with pandas and a SQL database
import pandas as pd
from sqlalchemy import create_engine
# my password is stored in a config.py file
from config import postgres_pwd

In [2]:
# create variables for the database and tables that we'll be using
db_name = "restaurant_db"
restaurant_table_name = "restaurant_info"
yelp_table_name = "yelp_rating"
cuisine_table_name = "cuisine_type"
google_table_name = "google_maps_rating"
trip_advisor_table_name = "trip_advisor_rating"

In [3]:
# Create restaurant database directly from Python
engine = create_engine(f"postgres://postgres:{postgres_pwd}@/postgres")
conn = engine.connect()
conn.execute("commit")
conn.execute(f"CREATE DATABASE {db_name}""")
conn.close()

In [4]:
# Create tables
connection_string = f"postgres:{postgres_pwd}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')
conn = engine.connect()
trans = conn.begin()

table_creation_sql = f"""
-- Drop table if exists
DROP TABLE IF EXISTS {restaurant_table_name};

-- Create the restaurant table
CREATE TABLE {restaurant_table_name} (
    restaurant_id INTEGER PRIMARY KEY NOT NULL,
    restaurant_name varchar,
    restaurant_website varchar,
    address1 varchar   NOT NULL,
    city varchar   NOT NULL,
    state varchar   NOT NULL,
    "zip code" varchar   NOT NULL
);

-- Drop table if exists
DROP TABLE IF EXISTS {yelp_table_name};

-- Create the yelp table
CREATE TABLE {yelp_table_name} (
    "restaurant_id" INTEGER PRIMARY KEY NOT NULL,
    "yelp_url" varchar   NOT NULL,
    "yelp_rating" varchar,
    "yelp_price_level" varchar,
    FOREIGN KEY (restaurant_id) REFERENCES {restaurant_table_name}(restaurant_id)
);

-- Drop table if exists
DROP TABLE IF EXISTS {cuisine_table_name};

-- Create the cuisine table
CREATE TABLE {cuisine_table_name} (
    "restaurant_id" int   NOT NULL,
    "cuisine" varchar   NOT NULL,
    FOREIGN KEY (restaurant_id) REFERENCES {restaurant_table_name}(restaurant_id)
);

-- Drop table if exists
DROP TABLE IF EXISTS {google_table_name};

-- Create the google table
CREATE TABLE {google_table_name} (
    "restaurant_id" INTEGER PRIMARY KEY NOT NULL,
    "google_maps_url" varchar   NOT NULL,
    "google_rating" varchar,
    "google_price_level" varchar,
    FOREIGN KEY (restaurant_id) REFERENCES {restaurant_table_name}(restaurant_id)
);

-- Drop table if exists
DROP TABLE IF EXISTS {trip_advisor_table_name};

-- Create the trip advisor table
CREATE TABLE {trip_advisor_table_name} (
    "restaurant_id" INTEGER PRIMARY KEY NOT NULL,
    "trip_advisor_rating" varchar,
    "trip_advisor_price_level" varchar,
    "trip_advisor_url" varchar NOT NULL,
    FOREIGN KEY (restaurant_id) REFERENCES {restaurant_table_name}(restaurant_id)
);
"""

conn.execute(table_creation_sql)

trans.commit()
conn.close() #it's good to be safe and close your connections when you're done with them


In [5]:
# read all the data from csvs
restaurant_df = pd.read_csv("../Resources/restaurant_final_data.csv", index_col=0)
yelp_df = pd.read_csv("../Resources/yelp_final_data.csv", index_col=0)
google_df = pd.read_csv("../Resources/google_final_data.csv", index_col=0)
tripadvisor_df = pd.read_csv("../Resources/tripadvisor_final_data.csv", index_col=0)
cuisine_df = pd.read_csv("../Resources/cuisine_data.csv", index_col=0)


In [6]:
# add data to restaurant table
conn = engine.connect()
restaurant_df.to_sql(name=restaurant_table_name, con=conn, if_exists='append', index=True)

In [7]:
sql_data_result = engine.execute(f"SELECT * FROM {restaurant_table_name}")

for sql_row in sql_data_result:
   print(sql_row)

sql_data_result.close()

(1, 'Rosie’s Trattoria', None, '1181 Sussex Tpke', 'Randolph', 'NJ', '7869')
(2, 'SubUrban Bar & Kitchen', 'sbknj.com', '500 NJ-10', 'Randolph', 'NJ', '7869')
(3, '4 Seasons Mediterranean Restaurant', '4seasonswharton.com', '322 S Main St', 'Wharton', 'NJ', '7885')
(4, 'Quiet Man Pub', 'quietmanpub.com', '64 E Mcfarlan St', 'Dover', 'NJ', '7801')
(5, 'El Lechon De Negron', None, '23 E Main St', 'Denville', 'NJ', '7834')
(6, 'The Corner Bistro', 'thecornerbistronj.com', '477 NJ-10', 'Randolph', 'NJ', '7869')
(7, 'ALEV Mediterranean Grill', 'alevmediterraneangrill.com', '76 Rte 46', 'Rockaway', 'NJ', '7866')
(8, 'Mr Crabby’s Craft Kitchen & Bar', 'mrcrabbys.com', '399 NJ-10', 'Randolph', 'NJ', '7869')
(9, 'Kabab Paradise', 'kababparadise.com', '124 Rte 10 W', 'Randolph', 'NJ', '7869')
(10, 'Takashi Japanese Cuisine', None, '23 Washington St', 'Morristown', 'NJ', '7960')
(11, 'Up Thai', None, '981 Tabor Rd', 'Morris Plains', 'NJ', '7950')
(12, 'The Windlass', 'thewindlass.com', '45 Nolans

In [8]:
# add data to yelp table
yelp_df.to_sql(name=yelp_table_name, con=conn, if_exists='append', index=True)

In [9]:
sql_data_result = engine.execute(f"SELECT * FROM {yelp_table_name}")

for sql_row in sql_data_result:
   print(sql_row)

sql_data_result.close()

(1, 'https://www.yelp.com/biz/rosie-s-trattoria-randolph?osq=Restaurants', '4.5', '$$$')
(2, 'https://www.yelp.com/biz/suburban-bar-and-kitchen-randolph?osq=Restaurants', '4.0', '$$')
(3, 'https://www.yelp.com/biz/4-seasons-mediterranean-restaurant-wharton?osq=Restaurants', '4.5', '$$')
(4, 'https://www.yelp.com/biz/quiet-man-pub-dover-2?osq=Restaurants', '4.5', '$$')
(5, 'https://www.yelp.com/biz/el-lechon-de-negron-denville?osq=Restaurants', '4.0', None)
(6, 'https://www.yelp.com/biz/the-corner-bistro-randolph-2?osq=Restaurants', '4.0', '$$')
(7, 'https://www.yelp.com/biz/alev-mediterranean-grill-rockaway-2?osq=Restaurants', '4.5', '$$')
(8, 'https://www.yelp.com/biz/mr-crabbys-craft-kitchen-and-bar-randolph-2?osq=Restaurants', '3.5', '$$')
(9, 'https://www.yelp.com/biz/kabab-paradise-randolph-2?osq=Restaurants', '4.5', '$$')
(10, 'https://www.yelp.com/biz/takashi-japanese-cuisine-morristown?osq=Restaurants', '5.0', None)
(11, 'https://www.yelp.com/biz/up-thai-morris-plains?osq=Resta

In [10]:
# add data to google table
google_df.to_sql(name=google_table_name, con=conn, if_exists='append', index=True)

In [11]:
sql_data_result = engine.execute(f"SELECT * FROM {yelp_table_name}")

for sql_row in sql_data_result:
   print(sql_row)

sql_data_result.close()

(1, 'https://www.yelp.com/biz/rosie-s-trattoria-randolph?osq=Restaurants', '4.5', '$$$')
(2, 'https://www.yelp.com/biz/suburban-bar-and-kitchen-randolph?osq=Restaurants', '4.0', '$$')
(3, 'https://www.yelp.com/biz/4-seasons-mediterranean-restaurant-wharton?osq=Restaurants', '4.5', '$$')
(4, 'https://www.yelp.com/biz/quiet-man-pub-dover-2?osq=Restaurants', '4.5', '$$')
(5, 'https://www.yelp.com/biz/el-lechon-de-negron-denville?osq=Restaurants', '4.0', None)
(6, 'https://www.yelp.com/biz/the-corner-bistro-randolph-2?osq=Restaurants', '4.0', '$$')
(7, 'https://www.yelp.com/biz/alev-mediterranean-grill-rockaway-2?osq=Restaurants', '4.5', '$$')
(8, 'https://www.yelp.com/biz/mr-crabbys-craft-kitchen-and-bar-randolph-2?osq=Restaurants', '3.5', '$$')
(9, 'https://www.yelp.com/biz/kabab-paradise-randolph-2?osq=Restaurants', '4.5', '$$')
(10, 'https://www.yelp.com/biz/takashi-japanese-cuisine-morristown?osq=Restaurants', '5.0', None)
(11, 'https://www.yelp.com/biz/up-thai-morris-plains?osq=Resta

In [12]:
# add data to trip advisor table
tripadvisor_df.to_sql(name=trip_advisor_table_name, con=conn, if_exists='append', index=True)

In [13]:
sql_data_result = engine.execute(f"SELECT * FROM {yelp_table_name}")

for sql_row in sql_data_result:
   print(sql_row)

sql_data_result.close()

(1, 'https://www.yelp.com/biz/rosie-s-trattoria-randolph?osq=Restaurants', '4.5', '$$$')
(2, 'https://www.yelp.com/biz/suburban-bar-and-kitchen-randolph?osq=Restaurants', '4.0', '$$')
(3, 'https://www.yelp.com/biz/4-seasons-mediterranean-restaurant-wharton?osq=Restaurants', '4.5', '$$')
(4, 'https://www.yelp.com/biz/quiet-man-pub-dover-2?osq=Restaurants', '4.5', '$$')
(5, 'https://www.yelp.com/biz/el-lechon-de-negron-denville?osq=Restaurants', '4.0', None)
(6, 'https://www.yelp.com/biz/the-corner-bistro-randolph-2?osq=Restaurants', '4.0', '$$')
(7, 'https://www.yelp.com/biz/alev-mediterranean-grill-rockaway-2?osq=Restaurants', '4.5', '$$')
(8, 'https://www.yelp.com/biz/mr-crabbys-craft-kitchen-and-bar-randolph-2?osq=Restaurants', '3.5', '$$')
(9, 'https://www.yelp.com/biz/kabab-paradise-randolph-2?osq=Restaurants', '4.5', '$$')
(10, 'https://www.yelp.com/biz/takashi-japanese-cuisine-morristown?osq=Restaurants', '5.0', None)
(11, 'https://www.yelp.com/biz/up-thai-morris-plains?osq=Resta

In [14]:
# add data to cuisine table
cuisine_df.to_sql(name=cuisine_table_name, con=conn, if_exists='append', index=True)

In [15]:
sql_data_result = engine.execute(f"SELECT * FROM {yelp_table_name}")

for sql_row in sql_data_result:
   print(sql_row)

sql_data_result.close()

(1, 'https://www.yelp.com/biz/rosie-s-trattoria-randolph?osq=Restaurants', '4.5', '$$$')
(2, 'https://www.yelp.com/biz/suburban-bar-and-kitchen-randolph?osq=Restaurants', '4.0', '$$')
(3, 'https://www.yelp.com/biz/4-seasons-mediterranean-restaurant-wharton?osq=Restaurants', '4.5', '$$')
(4, 'https://www.yelp.com/biz/quiet-man-pub-dover-2?osq=Restaurants', '4.5', '$$')
(5, 'https://www.yelp.com/biz/el-lechon-de-negron-denville?osq=Restaurants', '4.0', None)
(6, 'https://www.yelp.com/biz/the-corner-bistro-randolph-2?osq=Restaurants', '4.0', '$$')
(7, 'https://www.yelp.com/biz/alev-mediterranean-grill-rockaway-2?osq=Restaurants', '4.5', '$$')
(8, 'https://www.yelp.com/biz/mr-crabbys-craft-kitchen-and-bar-randolph-2?osq=Restaurants', '3.5', '$$')
(9, 'https://www.yelp.com/biz/kabab-paradise-randolph-2?osq=Restaurants', '4.5', '$$')
(10, 'https://www.yelp.com/biz/takashi-japanese-cuisine-morristown?osq=Restaurants', '5.0', None)
(11, 'https://www.yelp.com/biz/up-thai-morris-plains?osq=Resta

In [16]:
conn.close()